In [1]:
"""
Train the model using Yolov8 and Roboflow

Reference: https://blog.roboflow.com/how-to-train-yolov8-on-a-custom-dataset/
"""

'\nTrain the model using Yolov8 and Roboflow\n\nReference: https://blog.roboflow.com/how-to-train-yolov8-on-a-custom-dataset/\n'

In [3]:
!(nvcc --version) # Check if cuda is installed

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2025 NVIDIA Corporation
Built on Wed_Jan_15_19:20:09_PST_2025
Cuda compilation tools, release 12.8, V12.8.61
Build cuda_12.8.r12.8/compiler.35404655_0


In [4]:
!(nvidia-smi) # Check if cuda driver is working

Thu Jan 30 02:13:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.05              Driver Version: 560.35.05      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060 ...    Off |   00000000:01:00.0 Off |                  N/A |
| N/A   51C    P0             25W /   80W |      15MiB /   6144MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
# if not installed run: pip install ultralytics
import ultralytics
from torch import cuda
ultralytics.checks()
print("GPU is being used: ", cuda.is_available())

Ultralytics YOLOv8.1.29 🚀 Python-3.10.12 torch-2.2.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5834MiB)
Setup complete ✅ (16 CPUs, 14.9 GB RAM, 81.7/97.5 GB disk)
GPU is being used:  True


In [7]:
# if not installed run: pip install roboflow
# if you have a weird 'appengine' error run this: pip install urllib3==2.0.7
# other errors, run through chat gpt
# this was copied from exporting the dataset on roboflow with 'yolov8'

"""
Reference to Shape Dataset: https://universe.roboflow.com/arcturus-project/shape-data
"""

from roboflow import Roboflow
rf = Roboflow(api_key="O3voW9YlufPmeJibf4Jm") # Private Key for Team
project = rf.workspace("arcturus-project").project("shape-data")
version = project.version(8)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.29, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Shape-Data-8 in yolov8:: 100%|██████████| 30334/30334 [00:03<00:00, 8164.51it/s] 


In [ ]:
# Run the training of the model with the correct location to the data.yaml and custom settings (could take a while)
# This assumes data folder is in perception folder and this document is in perception folder
data_folder = ''

imgsz = 640

model= 'yolov8s.pt' 

# ! symbol lets you run command line interface commands (terminal commands) 
# Run this in command line if you get errors
# replace data.location with the name of the file folder
# WARNING: This will take a while and consume a lot of computer resources
!yolo task=detect \
    mode=train \
    model=yolov8s.pt \
    data={data.location}/data.yaml \
    epochs=25 \
    imgsz=640 \
    plots=True


In [ ]:
# validation of training results

!yolo task=detect \
    mode=val \
    model={HOME}/runs/detect/train/weights/best.pt \
    data={dataset.location}/data.yaml

In [ ]:
# predict using model

!yolo task=detect \
    mode=predict \
    model={HOME}/runs/detect/train/weights/best.pt \
    conf=0.25 \
    source={dataset.location}/test/images

In [2]:

from ultralytics import YOLO
model = YOLO("runs/detect/train_roboboat/weights/best.pt")

# results = model.predict(source="RoboBoat-Comp-2024-4/test/images")
results = model(["arcturus_test_image_1.jpg",
                 "arcturus_test_image_2.jpg",
                 "arcturus_test_image_3.jpg",
                 "arcturus_test_image_4.jpg"])
for i, result in enumerate(results):
    print(f"res = {result}")
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    result.save(filename=f'result_{i}.jpg')  # specify filename to save to disk


0: 480x640 1 red-buoy, 1 yellow-buoy, 1 yellow-duck, 26.3ms
1: 480x640 2 green-buoys, 1 red-buoy, 1 yellow-buoy, 26.3ms
2: 480x640 2 green-buoys, 1 red-buoy, 1 yellow-buoy, 1 yellow-duck, 26.3ms
3: 480x640 3 green-buoys, 1 red-buoy, 1 yellow-buoy, 26.3ms
Speed: 3.5ms preprocess, 26.3ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)
res = ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'black-buoy', 1: 'blue-buoy', 2: 'blue-cross', 3: 'green-buoy', 4: 'green-column-buoy', 5: 'red-buoy', 6: 'red-column-buoy', 7: 'red-dot', 8: 'yellow-buoy', 9: 'yellow-duck'}
obb: None
orig_img: array([[[ 35,   4, 213],
        [ 34,   3, 212],
        [ 37,   6, 215],
        ...,
        [ 62,  81, 102],
        [ 42,  61,  82],
        [ 60,  79, 100]],

       [[ 38,   7, 216],
        [ 36,   5, 214],
        [ 38,   7, 216],
        ...,
        [ 64,  83, 104],
        [ 60,  79, 100